In [193]:
print(f"Check the records if index has duplicate recores")

Check the records if index has duplicate recores


In [194]:
#!pip uninstall -y elasticsearch
#!pip install elasticsearch==7.13.0

In [195]:
# !pip freeze

In [196]:
!pip show elasticsearch

Name: elasticsearch
Version: 7.13.0
Summary: Python client for Elasticsearch
Home-page: https://github.com/elastic/elasticsearch-py
Author: Honza Král, Nick Lang
Author-email: honza.kral@gmail.com, nick@nicklang.com
License: Apache-2.0
Location: /home/biadmin/monitoring/jupyter_notebook/.venv/lib/python3.9/site-packages
Requires: certifi, urllib3
Required-by: 


In [197]:
from elasticsearch import Elasticsearch
import os
import json
import pandas as pd
from dotenv import load_dotenv

In [198]:
''' pip install python-dotenv'''
load_dotenv() # will search for .env file in local folder and load variables 

True

In [199]:
def get_headers():
    ''' Elasticsearch Header '''
    return {
            'Content-type': 'application/json', 
            'Authorization' : '{}'.format(os.getenv('BASIC_AUTH')),
            # 'Connection': 'close'
    }

In [200]:
def get_es_instance(host):
    es_client = Elasticsearch(hosts="http://{}".format(host), headers=get_headers(), timeout=5,  verify_certs=False)
    return es_client

# es_obj_s_client = get_es_instance("localhost:9200")

In [201]:
# resp = es_obj_s_client.cluster.health()
# print(json.dumps(resp, indent=2))

In [202]:
es_host_duplicates = {}

In [203]:
def check_duplicates_tasks(env):
    ''' check duplicates'''
    query = {
      "size": 0,
      "aggs": {
        "duplicates": {
          "terms": {
            "field": "TASKID",
            "min_doc_count": 2
            }
         }
        }
    }
    # print('\n', env)
    # print(os.getenv("{}_ES_HOST".format(str(env).upper())))
    es_host = "{}_ES_HOST".format(str(env).upper())
    # print(es_host)
    
    ''' instacne '''
    es_obj_s_client = get_es_instance(os.getenv(es_host))
    
    response = es_obj_s_client.search(index="wx_task", body=query)
    # print(json.dumps(response, indent=2))
    duplicates_list = response['aggregations']['duplicates']['buckets']
    # print(f"total duplicates : {json.dumps(len(duplicates_list), indent=2)}")
    # print(json.dumps(duplicates_list, indent=2))

    es_host_duplicates.update({es_host : len(duplicates_list)})

In [ ]:
env_list = ['prod1','prod2','prod3','prod4','prod6','prod7','prod8','prod9','prod10','prod12','prod13','prod14','prod16','prod17','prod18','prod19','prod20']
for env in env_list:
    check_duplicates_tasks(env)
print('\n Duplicate records')
print(json.dumps(es_host_duplicates, indent=2))